# Étape 1.1 – Préparation des données pour l'entrainement framewize (Etape 2) : 

**A FAIRE EN LOCAL**

Cette étape prépare toutes les données nécessaires pour l’entraînement frame-wise.
L’objectif est de :

1. Extraire 5 frames uniformément réparties par vidéo.

2. Nettoyer les images (floues, doublons).

3. Générer des splits 5-fold multi-label cohérents.

Toutes les opérations reposent sur les scripts présents dans src/preprocess/.

## 1 - Checking Preprocessing Scripts

Les scripts suivants doivent être présents :

- extract_5frames_uniform.py

- clean_frames_5.py

- make_splits_5fold.py

In [ ]:
import os
print(os.listdir("src/preprocessing"))

# 2 - Extracting 5 Uniform Frames per Video

Cette étape extrait cinq images par vidéo, réparties linéairement entre le début et la fin.
Les frames extraites sont sauvegardées dans :
**data/frames/5_forstep1and2/**

In [ ]:
RUN_EXTRACTION = True

if RUN_EXTRACTION:
    import subprocess, sys

    print(">> Extracting 5 frames per video...")
    result = subprocess.run(
        [sys.executable, "-m", "src.preprocessing.extract_5frames_uniform"],
        capture_output=True, text=True
    )
    print(result.stdout)
    print(result.stderr)
else:
    print("Extraction skipped.")


## 3 - Cleaning Extracted Frames

Nettoyage des données hors du dataset :

- les frames floues (mesure Laplacian),

- les doublons (perceptual hashing).

Les images supprimées sont placées dans : data/frames/5_forstep1and2_removed/

In [ ]:
RUN_CLEANING = True

if RUN_CLEANING:
    import subprocess, sys
    
    print(">> Cleaning frames (blurred + duplicates)...")
    result = subprocess.run(
        [sys.executable, "-m", "src.preprocess.clean_frames_5"],
        capture_output=True, text=True
    )
    print(result.stdout)
    print(result.stderr)
else:
    print("Cleaning skipped.")


## 4 - Generating 5-Fold Multi-label Splits

Cette partie crée une cross-validation 5-fold équilibrée pour un problème multi-label.
Les fichiers générés :

- data/splits/video_folds_5fold.csv

- data/splits/frames_5_forstep1and2_folds.csv

- reports/tables/preprocessing/class_distribution_per_fold.csv

In [ ]:
RUN_SPLITS = True

if RUN_SPLITS:
    import subprocess, sys

    print(">> Creating 5-fold stratified splits...")
    result = subprocess.run(
        [sys.executable, "-m", "src.preprocess.make_splits_5fold"],
        capture_output=True, text=True
    )
    print(result.stdout)
    print(result.stderr)
else:
    print("Split generation skipped.")


## 5 - Quick Verification of Outputs

On vérifie que les fichiers existent et que la répartition des folds est correcte.

In [ ]:
import pandas as pd
from pathlib import Path

frames_csv = Path("data/splits/frames_5_forstep1and2_folds.csv")
video_csv = Path("data/splits/video_folds_5fold.csv")

print("Frames CSV:", frames_csv.exists())
print("Videos CSV:", video_csv.exists())

if frames_csv.exists():
    df_frames = pd.read_csv(frames_csv)
    display(df_frames.head())
    print("\nFrames per fold:")
    print(df_frames["fold"].value_counts().sort_index())

if video_csv.exists():
    df_videos = pd.read_csv(video_csv)
    print("\nVideos per fold:")
    print(df_videos["fold"].value_counts().sort_index())
